Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [ ]:
# Standard library
import os

# Third-party packages
import h5py
import matplotlib.pyplot as pl
%matplotlib inline
import numpy as np
from IPython.display import display, Image
from skimage.io import imread
from sklearn.linear_model import LogisticRegression

# Python 2 & 3 support
from six.moves import cPickle as pickle
from six.moves.urllib.request import urlretrieve, urlopen

In [ ]:
data_URL = "https://s3.amazonaws.com/astronn/notMNIST.h5"
# cache_path = os.getcwd() # NOTE: if you change this, you must change it in all tutorial notebooks
cache_path = "/Users/adrian/projects/astronn/data"

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts in a series of 28 by 28 images. The labels simply identify the letter presented in each image (and are limited to A-J, so, 10 classes). The training set and test set have about 500000 and 19000 image-label pairs, respectively. Even with these sizes, it should be possible to train models quickly on any machine.

_Note: This could take some time! You are about to download a ~1.7 GB file. Go get some coffee._

In [ ]:
# how many bytes are we expecting
url = urlopen(data_URL)
meta = url.info()
expected_bytes = int(meta['Content-Length'])

cache_file = os.path.join(cache_path, os.path.basename(data_URL))
if (os.path.exists(cache_file) and os.stat(cache_file).st_size != expected_bytes) \
    or not os.path.exists(cache_file) or not os.path.isfile(cache_file):
    urlretrieve(data_URL, cache_file)
    
    received_bytes = os.stat(cache_file).st_size
    if received_bytes != expected_bytes:
        raise IOError("Download error: size expected = {} bytes, size received = {} bytes"
                      .format(expected_bytes, received_bytes))
        
    print("Data downloaded and verified.")

else:
    print("Data file already exists and is verified.")

---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font.  

Display a 3 by 3 grid of sample images that we just downloaded from the test set.

---

In [ ]:
with h5py.File(cache_file, 'r') as f:
    pass

In [ ]:
# Solution: 
fig,axes = pl.subplots(3,3,figsize=(5,5),sharex=True,sharey=True)
with h5py.File(cache_file, 'r') as f:
    for i in range(9):
        ax = axes.flat[i]
        ax.imshow(f['test']['images'][np.random.randint(f['test']['images'].shape[0])],
                  cmap='Greys', interpolation='nearest')

Now display the mean of all images from each class individually

In [ ]:
with h5py.File(cache_file, 'r') as f:
    pass

In [ ]:
# Solution: 
with h5py.File(cache_file, 'r') as f:
    # get a unique list of the classes
    classes = np.unique(f['test']['labels'])
    classes.sort()
    nclasses = len(classes)
    
    images = f['test']['images'][:]
    for i,cls in enumerate(classes):
        fig,ax = pl.subplots(1,1,figsize=(2,2))
        mean_img = images[f['test']['labels'][:] == cls].mean(axis=0) # select all images for a given class, take mean
        ax.imshow(mean_img, cmap='Greys', interpolation='nearest') # greyscale colormap, no interpolation

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [ ]:
np.random.seed(133)
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)

---
Problem 3
---------
Convince yourself that the data is still good after shuffling!

---

---
Problem 4
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [ ]:
np.histogram(train_labels, bins=np.arange(0,num_classes+1,1))

Prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune train_size as needed.

Also create a validation dataset for hyperparameter tuning.

In [ ]:
train_size = 500000
valid_size = 18000

valid_dataset = train_dataset[:valid_size,:,:]
valid_labels = train_labels[:valid_size]
train_dataset = train_dataset[valid_size:valid_size+train_size,:,:]
train_labels = train_labels[valid_size:valid_size+train_size]
print('Training', train_dataset.shape, train_labels.shape)
print('Validation', valid_dataset.shape, valid_labels.shape)

Finally, let's save the data for later reuse:

In [ ]:
pickle_file = 'notMNIST.pickle'

try:
    f = open(pickle_file, 'wb')
    save = {
        'train_dataset': train_dataset,
        'train_labels': train_labels,
        'valid_dataset': valid_dataset,
        'valid_labels': valid_labels,
        'test_dataset': test_dataset,
        'test_labels': test_labels,
    }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

In [ ]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.
Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [ ]:
overlaps = []
for test_img in test_dataset[:100]:
    diff = (train_dataset - test_img[None]).sum(axis=-1).sum(axis=-1)
    overlap = (diff == 0).sum()
    overlaps.append(overlap)

In [ ]:
print("Typical overlap:", np.median(overlaps))
pl.hist(overlaps)

---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [ ]:
model = LogisticRegression()

In [ ]:
subset = 5000
idx = np.random.choice(np.arange(train_dataset.shape[0]), size=subset)
train_subset_data = train_dataset[idx].reshape(subset, image_size*image_size)
train_subset_labels = train_labels[idx]

In [ ]:
model.fit(train_subset_data, train_subset_labels)

In [ ]:
predict_labels = model.predict(test_dataset.reshape(test_dataset.shape[0], image_size*image_size))

In [ ]:
(predict_labels != test_labels).sum() / test_labels.size